In [24]:
import pandas as pd

# reads in census data
csv_file_path = "NC_Census_2021/table01.csv"
df = pd.read_csv(csv_file_path)

# column names for age brackets of women
women_15_19_idxs = ["B01001_030E", "B01001_031E"]
women_20_24_idxs = ["B01001_032E", "B01001_033E", "B01001_034E"]
women_25_29_idxs = ["B01001_035E"]
women_30_49_idxs = ["B01001_036E", "B01001_037E", "B01001_038E", "B01001_039E"]

# ratios of women who report ever using emergency contraceptives (EC)
ec_px_15_19 = 0.205
ec_px_20_24 = 0.350
ec_px_25_29 = 0.358
ec_px_30_49 = 0.157

# holds the results
df.insert(0, "ec_risk", 0)

# adds up number of women per age group multiplied by ratio of EC usage
for columns, ratio in zip(
    [women_15_19_idxs, women_20_24_idxs, women_25_29_idxs, women_30_49_idxs], 
    [ec_px_15_19, ec_px_20_24, ec_px_25_29, ec_px_30_49]):
        df["ec_risk"] += df[columns].sum(axis=1) * ratio

# EC risk by county, and total number vulnerable
print(df["ec_risk"])
print(sum(df["ec_risk"]))

0       213.486
1       251.898
2       121.061
3       158.936
4       330.004
         ...   
2667    187.207
2668     90.056
2669    148.634
2670    143.469
2671    170.111
Name: ec_risk, Length: 2672, dtype: float64
525300.8609999998
